In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_autoinstaller
from time import sleep
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

In [31]:
sektorler = ["Bilgisayar Toptancılığı", "Boya", "Çimento", "Dayanıklı Tüketim", "Demir-Çelik Döküm", "Demir-Çelik Temel",
            "Deri Giyim", "Diğer", "Eğlence Hizmetleri", "Elektrik - Doğalgaz Dağıtım", "Elektrik Enerji Ürt",
            "Elektrik Üretim", "Endüstriyel Makine-Teçhizat Üretim", "Endüstriyel Tekstil","Gıda", "GYO", "Havayolları ve Hizm.", 
            "İletişim", "İletişim Cihazları", "İnşaat Malzemeleri", "İnşaat- Taahhüt", "Kablo", "Kağıt ürünleri", "Kırtasiye", "Kimyasal Ürün",
            "Madencilik", "Medya", "Meşrubat/İçecek", "Mobilya", "Otomotiv", "Otomotiv Lastiği", "Otomotiv Parçası", "Perakande-Ticaret",
            "Petrol", "Sağlık ve İlaç", "Savunma", "Seramik", "Tarım Kimyasalları", "Teknoloji", "Tekstil Entegre", "Truzim", "Ulaştırma-Lojistik"]

In [33]:
chromedriver_autoinstaller.install()

'C:\\Users\\okand\\anaconda3\\Lib\\site-packages\\chromedriver_autoinstaller\\142\\chromedriver.exe'

In [35]:
sector = "Elektrik Enerji"

In [37]:
def get_codes(sector):
    path = "C:\\Users\\okand\\anaconda3\\Lib\\site-packages\\chromedriver_autoinstaller\\142\\chromedriver.exe"
    service = webdriver.chrome.service.Service(path)
    service.start()
    options = Options()
    driver=webdriver.Chrome()

    url = "https://www.isyatirim.com.tr/tr-tr/analiz/hisse/Sayfalar/Temel-Degerler-Ve-Oranlar.aspx#page-5"
    driver.get(url)
    driver.maximize_window()

    sleep(3)

    driver.find_element(By.XPATH, '//*[@id="ctl00_ctl58_g_6c4325d1_d097_49fd_a0b9_121214c91535"]/div/div/div[1]/div[2]/div/div[1]').click()

    sleep(3)

    driver.find_element(By.XPATH, '//*[@id="ctl00_ctl58_g_6c4325d1_d097_49fd_a0b9_121214c91535"]/div/div/div[1]/div[2]/div/div[2]/span/span[1]/span/span[2]').click()

    sleep(3)

    search_box = driver.find_element(By.XPATH,'/html/body/span/span/span[1]/input')
    search_box.send_keys("Ulusal-Tum" + Keys.ENTER)

    sleep(3)

    driver.find_element(By.XPATH, '//*[@id="ctl00_ctl58_g_6c4325d1_d097_49fd_a0b9_121214c91535"]/div/div/div[1]/div[1]/div/div[1]').click()

    sleep(3)

    driver.find_element(By.XPATH, '//*[@id="ctl00_ctl58_g_6c4325d1_d097_49fd_a0b9_121214c91535"]/div/div/div[1]/div[1]/div/div[2]/span/span[1]/span/span[2]').click()

    sleep(3)

    search_box = driver.find_element(By.XPATH,"/html/body/span/span/span[1]/input")
    sleep(2)
    search_box.send_keys(sector + Keys.ENTER)
    sleep(3)

    names=driver.find_elements(By.CLASS_NAME,"sorting_1") # liste

    sleep(2)
    new_list=[]
    for i in range(len(names)):
        new_list.append(names[i].text)

    empty_list=[""]
    new_list = [x for x in new_list if x not in empty_list]

    sleep(2)

    driver.quit()

    df = pd.DataFrame(new_list, columns=["Codes"])
    df.to_excel(f"{sector} şirketleri.xlsx", index=False)

    return df

In [39]:
data = get_codes(sector)
data

,Codes
0,ALFAS
1,ASTOR
2,CWENE
3,EMKEL
4,EUPWR
5,GESAN
6,KLYPV
7,KONTR
8,MAGEN
9,MOGAN


In [40]:
def get_financial_statement(data):
    financial_dir={}
    hisseler=data.Codes.values.tolist()
    
    for x in hisseler:
        hisse=x
        tarihler=[]
        yıllar=[]
        donemler=[]
        grup=[]
    
        url1="https://www.isyatirim.com.tr/tr-tr/analiz/hisse/Sayfalar/sirket-karti.aspx?hisse="+hisse
        r1=requests.get(url1)
        soup=BeautifulSoup(r1.text,"html.parser")
        secim=soup.find("select",id="ddlMaliTabloFirst")
        secim2=soup.find("select",id="ddlMaliTabloGroup")
        
        try:
            cocuklar=secim.findChildren("option")
            grup=secim2.find("option")["value"]
            
            for i in cocuklar:
                tarihler.append(i.string.rsplit("/"))
            for j in tarihler:
                yıllar.append(j[0])
                donemler.append(j[1])
    
            if len(tarihler)>=4:
                parametreler=(
                    ("companyCode",hisse),
                    ("exchange","TRY"), #Eğer dolar bazlı istiyorsanız "TRY" yerine "USD" yazınız
                    ("financialGroup",grup),
                    ("year1",yıllar[0]),
                    ("period1",donemler[0]),
                    ("year2",yıllar[1]),
                    ("period2",donemler[1]),
                    ("year3",yıllar[2]),
                    ("period3",donemler[2]),
                    ("year4",yıllar[3]),
                    ("period4",donemler[3]))
                url2="https://www.isyatirim.com.tr/_layouts/15/IsYatirim.Website/Common/Data.aspx/MaliTablo"
                r2=requests.get(url2,params=parametreler).json()["value"]
                veri=pd.DataFrame.from_dict(r2)
                veri.drop(columns=["itemCode","itemDescEng"],inplace=True)           
            else:
                continue
        except AttributeError:
            continue
        
        del tarihler[0:4]
        tumveri=[veri]
            
        for _ in range(0,int(len(tarihler)+1)):
            if len(tarihler)==len(yıllar):
                del tarihler[0:4]
            else:
                yıllar=[]
                donemler=[]
                for j in tarihler:
                    yıllar.append(j[0])
                    donemler.append(j[1])
                
                if len(tarihler)>=4:
                    parametreler2=(
                    ("companyCode",hisse),
                    ("exchange","TRY"), #Eğer dolar bazlı istiyorsanız "TRY" yerine "USD" yazınız
                    ("financialGroup",grup),
                    ("year1",yıllar[0]),
                    ("period1",donemler[0]),
                    ("year2",yıllar[1]),
                    ("period2",donemler[1]),
                    ("year3",yıllar[2]),
                    ("period3",donemler[2]),
                    ("year4",yıllar[3]),
                    ("period4",donemler[3]))
                    r3=requests.get(url2,params=parametreler2).json()["value"]
                    veri2=pd.DataFrame.from_dict(r3)
                    try:
                        veri2.drop(columns=["itemCode","itemDescTr","itemDescEng"],inplace=True)
                        tumveri.append(veri2)
                    except KeyError:
                        continue
        veri3=pd.concat(tumveri,axis=1)
        baslık=["Bilanço"]
        for i in cocuklar:
            baslık.append(i.string)
    
        baslıkfark=len(baslık)-len(veri3.columns)
    
        if baslıkfark!=0:
            del baslık[-baslıkfark:]
    
        veri3=veri3.set_axis(baslık,axis=1)
        veri3[baslık[1:]]=veri3[baslık[1:]].astype(float)
        veri3=veri3.fillna(0)
        veri3.set_index("Bilanço", inplace=True)
        veri3.loc["FAVÖK",:] = veri3.loc["Net Faaliyet Kar/Zararı",:] + veri3.loc["Amortisman Giderleri",:]
        financial_dir[x]=veri3
    return financial_dir

In [41]:
donem = input("Son Dönemi giriniz: ")
donem

Son Dönemi giriniz:  2025/9


'2025/9'

In [45]:
import yfinance as yf

In [52]:
def create_table(data, donem):
    pd.set_option('display.float_format', '{:.2f}'.format)

    financial_dir = get_financial_statement(data)
    analiz = pd.DataFrame(index = financial_dir.keys())
    for i in financial_dir.keys():
        try:
            analiz.loc[i, "Özkaynaklar"]=financial_dir[i].loc["Özkaynaklar", donem]
        except KeyError:
            print(f"{i} hissesinin bilançosunun yeni dönem bilançosu açıklanmamış.")
    
    analiz.dropna(inplace = True)
    
    for i in analiz.index.tolist():
        analiz.loc[i, "Ödenmiş Sermaye"]=financial_dir[i].loc["  Ödenmiş Sermaye", donem]
        analiz.loc[i, "Net Borç"]=(financial_dir[i].iloc[31,0] + financial_dir[i].iloc[32,0] + financial_dir[i].iloc[45,0]) -\
        (financial_dir[i].iloc[1,0]+financial_dir[i].iloc[2,0])
        price = yf.download(f"{i}.IS",start="2025-05-01")["Close"].iloc[-1,-1]
        analiz.loc[i,"Fiyat"] = price
        analiz.loc[i, "PD"] = analiz.loc[i,"Fiyat"] * analiz.loc[i, "Ödenmiş Sermaye"]
        analiz.loc[i, "FD"] = analiz.loc[i, "PD"] + analiz.loc[i, "Net Borç"]
        if donem.split("/")[1] in  ["3","6","9"]:
            donem_onceki = "202"+str(int(donem.split("/")[0][-1])-1)+"/"+donem.split("/")[1]
            donem_yilsonu= "202"+str(int(donem.split("/")[0][-1])-1)+"/12"
            
            analiz.loc[i, "Satış Gelirleri"]=financial_dir[i].loc["Satış Gelirleri", donem] + \
            financial_dir[i].loc["Satış Gelirleri", donem_yilsonu] - \
            financial_dir[i].loc["Satış Gelirleri", donem_onceki]
            
            analiz.loc[i, "FAVÖK"]=financial_dir[i].loc["FAVÖK", donem] + \
            financial_dir[i].loc["FAVÖK", donem_yilsonu] - \
            financial_dir[i].loc["FAVÖK", donem_onceki]
            
            analiz.loc[i, "Net Kar"]=financial_dir[i].loc["Ana Ortaklık Payları", donem] + \
            financial_dir[i].loc["Ana Ortaklık Payları", donem_yilsonu] - \
            financial_dir[i].loc["Ana Ortaklık Payları", donem_onceki]
        else:
            analiz.loc[i, "Satış Gelirleri"]=financial_dir[i].loc["Satış Gelirleri", donem]
            analiz.loc[i, "FAVÖK"]=financial_dir[i].loc["FAVÖK", donem]
            analiz.loc[i, "Net Kar"]=financial_dir[i].loc["Ana Ortaklık Payları", donem]
    analiz.to_excel(f"C:/Users/okand/OneDrive/Desktop/Borsa/{sector}.xlsx", index=True)
    return analiz
        

In [54]:
df = create_table(data, donem)
df

ULUSE hissesinin bilançosunun yeni dönem bilançosu açıklanmamış.


C:\Users\okand\AppData\Local\Temp\ipykernel_11372\3635971931.py:18: FutureWarning: YF.download() has changed argument auto_adjust default to True
  price = yf.download(f"{i}.IS",start="2025-05-01")["Close"].iloc[-1,-1]
[*********************100%***********************]  1 of 1 completed
C:\Users\okand\AppData\Local\Temp\ipykernel_11372\3635971931.py:18: FutureWarning: YF.download() has changed argument auto_adjust default to True
  price = yf.download(f"{i}.IS",start="2025-05-01")["Close"].iloc[-1,-1]
[*********************100%***********************]  1 of 1 completed
C:\Users\okand\AppData\Local\Temp\ipykernel_11372\3635971931.py:18: FutureWarning: YF.download() has changed argument auto_adjust default to True
  price = yf.download(f"{i}.IS",start="2025-05-01")["Close"].iloc[-1,-1]
[*********************100%***********************]  1 of 1 completed
C:\Users\okand\AppData\Local\Temp\ipykernel_11372\3635971931.py:18: FutureWarning: YF.download() has changed argument auto_adjust defaul

,Özkaynaklar,Ödenmiş Sermaye,Net Borç,Fiyat,PD,FD,Satış Gelirleri,FAVÖK,Net Kar
ALFAS,5319195074.00,368000000.00,1725748050.00,41.80,15382399719.24,17108147769.24,8441453974.00,238468851.00,-93815380.00
ASTOR,29305773835.00,998000000.00,-5549807146.00,91.75,91566500000.00,86016692854.00,30844022932.00,9141463778.00,6441361229.00
CWENE,12850924322.00,1000000000.00,6825793369.00,26.00,26000000000.00,32825793369.00,14574481942.00,3155040057.00,1377326775.00
EMKEL,977944082.00,150000000.00,420207254.00,53.65,8047500228.88,8467707482.88,907387450.00,-1169563.00,-21220461.00
EUPWR,10871430000.00,660000000.00,1467888000.00,28.78,18994800453.19,20462688453.19,10142985136.00,2006193148.00,725798208.00
GESAN,18237661000.00,460000000.00,2228532000.00,43.26,19899599227.91,22128131227.91,20369944565.00,4155052037.00,1187062997.00
KLYPV,14144376522.00,411441018.00,13505781102.00,53.75,22114954717.50,35620735819.50,8742848934.00,1122073029.00,-188645784.00
KONTR,6859236219.00,650000000.00,12068079456.00,34.40,22360000991.82,34428080447.82,14545934168.00,624384542.00,-1275827143.00
MAGEN,14668779532.00,1180000000.00,4068845100.00,32.36,38184800720.21,42253645820.21,1626624808.00,459988255.00,-1243323740.00
MOGAN,79521211999.00,2440108000.00,31951959631.00,8.28,20204093588.42,52156053219.42,12990797377.00,6866068145.00,-9663610318.00


In [55]:
df.to_excel(f"C:/Users/okand/OneDrive/Desktop/Borsa/{sector}.xlsx", index=True)